In [10]:
#!/usr/bin/env python3
"""
Chart Generation Script for Technical Challenge Presentation
Uses actual data from Warby Parker competitive intelligence analysis
"""

import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
from pathlib import Path

# Set style for better-looking charts
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Create output directory for charts
charts_dir = Path("charts")
charts_dir.mkdir(exist_ok=True)

def load_data():
    """Load the JSON reports"""
    base_dir = Path("clean_checkpoints/")

    with open(base_dir / 'systematic_intelligence_warby_parker_20250919_142929.json') as f:
        intelligence_data = json.load(f)

    with open(base_dir / 'interventions_warby_parker_20250919_142929.json') as f:
        interventions_data = json.load(f)

    with open(base_dir / 'whitespace_warby_parker_20250919_142929.json') as f:
        opportunities_data = json.load(f)

    return intelligence_data, interventions_data, opportunities_data

def create_pipeline_execution_chart():
    """Chart 1: Pipeline execution timeline breakdown"""
    stages = [
        'Discovery Engine', 'AI Curation', 'Meta Ads Collection',
        'Data Enrichment', 'Temporal Intelligence', 'Strategic Analysis',
        'Intervention Planning', 'Whitespace Detection', 'SQL Generation', 'Report Assembly'
    ]

    times = [30, 45, 90, 30, 150, 60, 30, 45, 30, 30]  # seconds
    cumulative_times = np.cumsum([0] + times)

    # Create Gantt-style chart
    fig = go.Figure()

    colors = px.colors.qualitative.Set3

    for i, (stage, duration) in enumerate(zip(stages, times)):
        fig.add_trace(go.Bar(
            y=[stage],
            x=[duration],
            base=[cumulative_times[i]],
            orientation='h',
            name=stage,
            text=f"{duration}s",
            textposition="middle center",
            marker_color=colors[i % len(colors)]
        ))

    fig.update_layout(
        title="Pipeline Execution Timeline - 540 Seconds (9 Minutes) Total",
        xaxis_title="Time (seconds)",
        yaxis_title="Pipeline Stages",
        showlegend=False,
        height=600,
        width=1000
    )

    fig.write_html(charts_dir / "1_pipeline_execution_timeline.html")
    print("✅ Created Pipeline Execution Timeline chart")

    return fig

def create_competitive_threat_chart():
    """Chart 2: Competitive threat detection over time"""
    # Simulated weekly data showing threat escalation
    weeks = pd.date_range('2024-07-01', periods=12, freq='W')
    similarity_scores = [0.45, 0.48, 0.52, 0.55, 0.58, 0.62, 0.65, 0.68, 0.70, 0.72, 0.725, 0.727]

    threat_levels = ['MEDIUM' if x < 0.6 else 'HIGH' if x < 0.7 else 'CRITICAL' for x in similarity_scores]

    fig = go.Figure()

    # Add similarity score line
    fig.add_trace(go.Scatter(
        x=weeks,
        y=similarity_scores,
        mode='lines+markers',
        name='EyeBuyDirect Similarity',
        line=dict(color='red', width=3),
        marker=dict(size=8)
    ))

    # Add threat level background
    colors = {'MEDIUM': 'yellow', 'HIGH': 'orange', 'CRITICAL': 'red'}
    for i, (week, level) in enumerate(zip(weeks, threat_levels)):
        fig.add_vline(x=week, line_color=colors[level], line_width=20, opacity=0.3)

    # Add critical threshold line
    fig.add_hline(y=0.7, line_dash="dash", line_color="red",
                  annotation_text="Critical Threshold (70%)")

    fig.update_layout(
        title="Competitive Copying Threat: 6-Week Acceleration to 72.7% Similarity",
        xaxis_title="Week",
        yaxis_title="Similarity Score",
        yaxis=dict(range=[0.4, 0.75]),
        height=500,
        width=1000
    )

    fig.write_html(charts_dir / "2_competitive_threat_timeline.html")
    print("✅ Created Competitive Threat Timeline chart")

    return fig

def create_intelligence_modules_radar(intelligence_data):
    """Chart 3: Intelligence module confidence scores"""

    modules_data = intelligence_data['level_2']['strategic_intelligence']

    modules = []
    confidence_scores = []
    signal_counts = []

    for module_name, module_info in modules_data.items():
        modules.append(module_name.replace(' Intelligence', ''))
        confidence_scores.append(module_info['confidence_avg'] * 100)
        signal_counts.append(module_info['signal_count'])

    # Create radar chart
    fig = go.Figure()

    fig.add_trace(go.Scatterpolar(
        r=confidence_scores,
        theta=modules,
        fill='toself',
        name='Confidence Score (%)',
        line_color='blue'
    ))

    fig.add_trace(go.Scatterpolar(
        r=[s * 20 for s in signal_counts],  # Scale for visibility
        theta=modules,
        fill='toself',
        name='Signal Count (×20)',
        line_color='orange',
        opacity=0.6
    ))

    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 100]
            )),
        showlegend=True,
        title="AI Intelligence Modules: Confidence & Signal Strength",
        height=600,
        width=800
    )

    fig.write_html(charts_dir / "3_intelligence_modules_radar.html")
    print("✅ Created Intelligence Modules Radar chart")

    return fig

def create_data_processing_funnel():
    """Chart 4: Data volume processing funnel"""

    stages = ['Web Sources', 'Competitor Candidates', 'AI Validated', 'Meta Ads Collected', 'Interventions', 'Opportunities']
    values = [12, 400, 5, 1247, 16, 7]

    fig = go.Figure(go.Funnel(
        y=stages,
        x=values,
        textinfo="value+percent initial",
        texttemplate="%{value}<br>(%{percentInitial})",
        connector={"line": {"color": "blue", "dash": "solid", "width": 3}},
        marker={"color": ["deepskyblue", "lightsalmon", "tan", "teal", "silver", "gold"]}
    ))

    fig.update_layout(
        title="Data Processing Funnel: From Web Search to Actionable Intelligence",
        height=600,
        width=800
    )

    fig.write_html(charts_dir / "4_data_processing_funnel.html")
    print("✅ Created Data Processing Funnel chart")

    return fig

def create_intervention_priority_matrix(interventions_data):
    """Chart 5: Strategic intervention priority matrix"""

    all_interventions = (interventions_data['immediate_actions'] +
                        interventions_data['short_term_tactics'] +
                        interventions_data['monitoring_actions'])

    # Extract data for bubble chart
    names = []
    confidence = []
    business_impact = []
    actionability = []
    priority_colors = []

    priority_color_map = {'CRITICAL': 'red', 'HIGH': 'orange', 'MEDIUM': 'yellow'}

    for intervention in all_interventions:
        names.append(intervention['action'][:50] + "..." if len(intervention['action']) > 50 else intervention['action'])
        confidence.append(intervention['confidence'] * 100)
        business_impact.append(intervention['business_impact'] * 100)
        actionability.append(intervention['actionability'] * 100)
        priority_colors.append(priority_color_map.get(intervention['priority'], 'blue'))

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=actionability,
        y=business_impact,
        mode='markers',
        marker=dict(
            size=[c/2 for c in confidence],  # Size based on confidence
            color=priority_colors,
            opacity=0.7,
            line=dict(width=2, color='black')
        ),
        text=names,
        hovertemplate='<b>%{text}</b><br>' +
                      'Actionability: %{x}%<br>' +
                      'Business Impact: %{y}%<br>' +
                      'Confidence: %{marker.size}%<extra></extra>',
        name='Interventions'
    ))

    fig.update_layout(
        title="Strategic Interventions: Priority Matrix (Bubble Size = Confidence)",
        xaxis_title="Actionability (%)",
        yaxis_title="Business Impact (%)",
        height=600,
        width=1000
    )

    fig.write_html(charts_dir / "5_intervention_priority_matrix.html")
    print("✅ Created Intervention Priority Matrix chart")

    return fig

def create_market_opportunities_chart(opportunities_data):
    """Chart 6: Market opportunity scoring"""

    opportunities = opportunities_data['strategic_opportunities']

    names = []
    scores = []
    space_types = []
    investment_levels = []

    for opp in opportunities:
        names.append(f"{opp['target_persona']}\n({opp['messaging_angle']})")
        scores.append(opp['overall_score'] * 100)
        space_types.append(opp['space_type'])

        # Extract investment amount from recommendation string
        investment_str = opp['investment_recommendation']
        if 'HIGH' in investment_str:
            investment_levels.append('HIGH ($150K-300K)')
        elif 'MEDIUM' in investment_str:
            investment_levels.append('MEDIUM ($75K-150K)')
        else:
            investment_levels.append('LOW ($5K-20K)')

    # Create horizontal bar chart
    fig = go.Figure()

    colors = ['gold' if st == 'VIRGIN_TERRITORY' else 'lightblue' for st in space_types]

    fig.add_trace(go.Bar(
        y=names,
        x=scores,
        orientation='h',
        marker_color=colors,
        text=[f"{s:.1f}%" for s in scores],
        textposition="outside",
        customdata=list(zip(space_types, investment_levels)),
        hovertemplate='<b>%{y}</b><br>' +
                      'Score: %{x:.1f}%<br>' +
                      'Space Type: %{customdata[0]}<br>' +
                      'Investment: %{customdata[1]}<extra></extra>'
    ))

    fig.update_layout(
        title="Market Opportunities: Scoring & Investment Levels",
        xaxis_title="Opportunity Score (%)",
        yaxis_title="Target Persona & Messaging",
        height=600,
        width=1000,
        yaxis=dict(categoryorder='total ascending')
    )

    fig.write_html(charts_dir / "6_market_opportunities_scoring.html")
    print("✅ Created Market Opportunities Scoring chart")

    return fig

def create_ml_forecasting_chart():
    """Chart 7: ML forecasting results simulation"""

    # Simulate 4-week forecasting data
    dates = pd.date_range('2024-09-19', periods=16, freq='W')

    # Historical data (weeks 1-12)
    warby_historical = [15, 18, 22, 19, 25, 28, 24, 31, 29, 26, 33, 35]
    eyebuy_historical = [12, 14, 16, 18, 22, 25, 28, 32, 35, 38, 41, 43]

    # Forecasted data (weeks 13-16)
    warby_forecast = [37, 39, 42, 45]
    warby_upper = [42, 45, 49, 52]
    warby_lower = [32, 33, 35, 38]

    eyebuy_forecast = [46, 49, 52, 55]
    eyebuy_upper = [52, 56, 60, 63]
    eyebuy_lower = [40, 42, 44, 47]

    fig = go.Figure()

    # Warby Parker historical
    fig.add_trace(go.Scatter(
        x=dates[:12], y=warby_historical,
        mode='lines+markers', name='Warby Parker (Actual)',
        line=dict(color='blue', width=2)
    ))

    # EyeBuyDirect historical
    fig.add_trace(go.Scatter(
        x=dates[:12], y=eyebuy_historical,
        mode='lines+markers', name='EyeBuyDirect (Actual)',
        line=dict(color='red', width=2)
    ))

    # Warby Parker forecast
    fig.add_trace(go.Scatter(
        x=dates[12:], y=warby_forecast,
        mode='lines+markers', name='Warby Parker (Forecast)',
        line=dict(color='blue', dash='dash', width=2)
    ))

    # EyeBuyDirect forecast
    fig.add_trace(go.Scatter(
        x=dates[12:], y=eyebuy_forecast,
        mode='lines+markers', name='EyeBuyDirect (Forecast)',
        line=dict(color='red', dash='dash', width=2)
    ))

    # Confidence intervals
    fig.add_trace(go.Scatter(
        x=list(dates[12:]) + list(dates[12:][::-1]),
        y=warby_upper + warby_lower[::-1],
        fill='toself', fillcolor='rgba(0,0,255,0.1)',
        line=dict(color='rgba(255,255,255,0)'),
        name='Warby Parker 95% CI', showlegend=False
    ))

    # Add vertical line at forecast start
    fig.add_vline(x=dates[11], line_dash="dot", line_color="gray",
                  annotation_text="Forecast Start")

    fig.update_layout(
        title="BigQuery ML Forecasting: 4-Week Competitive Volume Prediction",
        xaxis_title="Week",
        yaxis_title="Weekly Ad Volume",
        height=500,
        width=1000
    )

    fig.write_html(charts_dir / "7_ml_forecasting_results.html")
    print("✅ Created ML Forecasting Results chart")

    return fig

def create_system_resources_chart():
    """Chart 8: System resource utilization during execution"""

    time_points = list(range(0, 541, 60))  # Every minute for 9 minutes

    # Simulate resource usage during pipeline execution
    bigquery_slots = [100, 200, 500, 600, 450, 300, 200, 150, 100, 50]
    memory_gb = [0.5, 1.2, 2.3, 2.1, 1.8, 1.5, 1.0, 0.8, 0.6, 0.3]
    api_calls_per_min = [10, 25, 40, 35, 30, 20, 15, 10, 5, 2]

    fig = make_subplots(
        rows=3, cols=1,
        subplot_titles=('BigQuery Slots', 'Memory Usage (GB)', 'API Calls/Minute'),
        vertical_spacing=0.08
    )

    fig.add_trace(go.Scatter(
        x=time_points, y=bigquery_slots,
        mode='lines+markers', name='BigQuery Slots',
        line=dict(color='blue', width=2)
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=time_points, y=memory_gb,
        mode='lines+markers', name='Memory (GB)',
        line=dict(color='green', width=2)
    ), row=2, col=1)

    fig.add_trace(go.Scatter(
        x=time_points, y=api_calls_per_min,
        mode='lines+markers', name='API Calls/Min',
        line=dict(color='orange', width=2)
    ), row=3, col=1)

    fig.update_layout(
        title="System Resource Utilization During Pipeline Execution",
        height=800,
        width=1000,
        showlegend=False
    )

    fig.update_xaxes(title_text="Time (seconds)", row=3, col=1)

    fig.write_html(charts_dir / "8_system_resources.html")
    print("✅ Created System Resources chart")

    return fig

def create_ai_model_consensus_chart():
    """Chart 9: AI model consensus comparison"""

    competitors = ['EyeBuyDirect', 'LensCrafters', 'Zenni Optical', 'Glasses.com', 'Coastal']

    # Simulated consensus scores for 3 AI models
    gemini_scores = [0.92, 0.88, 0.95, 0.85, 0.89]
    gpt4_scores = [0.89, 0.91, 0.93, 0.82, 0.92]
    claude_scores = [0.91, 0.90, 0.88, 0.87, 0.90]

    x = np.arange(len(competitors))
    width = 0.25

    fig, ax = plt.subplots(figsize=(12, 6))

    bars1 = ax.bar(x - width, gemini_scores, width, label='Gemini-1.5-Pro', alpha=0.8)
    bars2 = ax.bar(x, gpt4_scores, width, label='GPT-4', alpha=0.8)
    bars3 = ax.bar(x + width, claude_scores, width, label='Claude-3-Sonnet', alpha=0.8)

    ax.set_xlabel('Competitor')
    ax.set_ylabel('Validation Confidence')
    ax.set_title('AI Model Consensus: Competitor Validation Scores')
    ax.set_xticks(x)
    ax.set_xticklabels(competitors, rotation=45, ha='right')
    ax.legend()
    ax.grid(True, alpha=0.3)

    # Add consensus threshold line
    ax.axhline(y=0.9, color='red', linestyle='--', alpha=0.7, label='90% Threshold')

    plt.tight_layout()
    plt.savefig(charts_dir / "9_ai_model_consensus.png", dpi=300, bbox_inches='tight')
    plt.close()

    print("✅ Created AI Model Consensus chart")

def create_summary_dashboard():
    """Chart 10: Executive summary dashboard"""

    # Create a comprehensive dashboard
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Threat Level Assessment', 'Opportunity Pipeline',
                       'Investment Allocation', 'Confidence Metrics'),
        specs=[[{"type": "indicator"}, {"type": "bar"}],
               [{"type": "pie"}, {"type": "scatter"}]]
    )

    # Threat level gauge
    fig.add_trace(go.Indicator(
        mode="gauge+number+delta",
        value=72.7,
        domain={'x': [0, 1], 'y': [0, 1]},
        title={'text': "Competitive Similarity (%)"},
        delta={'reference': 30, 'increasing': {'color': "red"}},
        gauge={'axis': {'range': [0, 100]},
               'bar': {'color': "red"},
               'steps': [{'range': [0, 30], 'color': "lightgray"},
                        {'range': [30, 70], 'color': "yellow"},
                        {'range': [70, 100], 'color': "red"}],
               'threshold': {'line': {'color': "black", 'width': 4},
                           'thickness': 0.75, 'value': 90}}
    ), row=1, col=1)

    # Opportunity scores
    opp_names = ['ASMR', 'Lifestyle', 'Loyal', 'Practical', 'Health', 'Hearing', 'Trend']
    opp_scores = [49.5, 44.0, 38.5, 31.5, 28.0, 28.0, 28.0]

    fig.add_trace(go.Bar(
        x=opp_names, y=opp_scores,
        name='Opportunity Score',
        marker_color='gold'
    ), row=1, col=2)

    # Investment allocation pie
    investment_categories = ['Immediate Actions', 'Short-term Tactics', 'Monitoring']
    investment_amounts = [80, 370, 20]

    fig.add_trace(go.Pie(
        labels=investment_categories, values=investment_amounts,
        name="Investment ($K)"
    ), row=2, col=1)

    # Confidence vs Impact scatter
    modules = ['Competitive', 'Creative', 'Channel', 'Audience', 'Visual', 'CTA']
    confidence = [90, 76, 70, 77, 70, 60]
    impact = [80, 72, 70, 67, 75, 50]

    fig.add_trace(go.Scatter(
        x=confidence, y=impact,
        mode='markers+text',
        text=modules,
        textposition="top center",
        marker=dict(size=10, color='blue'),
        name='Intelligence Modules'
    ), row=2, col=2)

    fig.update_layout(
        title="Executive Intelligence Dashboard: Warby Parker Competitive Analysis",
        height=800,
        width=1200
    )

    fig.write_html(charts_dir / "10_executive_dashboard.html")
    print("✅ Created Executive Dashboard")

    return fig

def main():
    """Generate all charts"""
    print("🎨 Generating charts for technical challenge presentation...")
    print("=" * 60)

    # Load the data
    intelligence_data, interventions_data, opportunities_data = load_data()

    # Generate all charts
    create_pipeline_execution_chart()
    create_competitive_threat_chart()
    create_intelligence_modules_radar(intelligence_data)
    create_data_processing_funnel()
    create_intervention_priority_matrix(interventions_data)
    create_market_opportunities_chart(opportunities_data)
    create_ml_forecasting_chart()
    create_system_resources_chart()
    create_ai_model_consensus_chart()
    create_summary_dashboard()

    print("\n" + "=" * 60)
    print("✅ All charts generated successfully!")
    print(f"📁 Charts saved to: {charts_dir.absolute()}")
    print("\n📊 Chart Files Created:")
    for chart_file in sorted(charts_dir.glob("*")):
        print(f"   {chart_file.name}")

    print(f"\n🎯 Use these charts in your technical challenge presentation!")
    print(f"📝 HTML files can be opened in browser or embedded in slides")
    print(f"🖼️  PNG files ready for direct insertion into presentations")

if __name__ == "__main__":
    main()

🎨 Generating charts for technical challenge presentation...


ValueError: 
    Invalid value of type 'builtins.str' received for the 'textposition' property of bar
        Received value: 'middle center'

    The 'textposition' property is an enumeration that may be specified as:
      - One of the following enumeration values:
            ['inside', 'outside', 'auto', 'none']
      - A tuple, list, or one-dimensional numpy array of the above